In [5]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import Bio

In [6]:
# import data from computer
path: str = './data/' #folder where files are stored
species: pd.DataFrame = pd.read_csv(os.path.join(path, "cross-species.csv"), dtype=str)   # imports file from the folder as species, based on name
identifiers: pd.DataFrame = pd.read_csv(os.path.join(path, "identifiers.tsv"), sep='\t', dtype=str) # imports 3rd file from the folder, based on name

#import fasta file with biopython (Bio)
from Bio import SeqIO
fasta_id = []
fasta_seq = []
with open(os.path.join(path, "identifiers.fasta"), 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"): 
        fasta_id.append(record.id) 
        fasta_seq.append(str(record.seq))
# Create df prot_seq which includes proper ID and sequences
fasta_id_df = pd.DataFrame([item.split('|') for item in fasta_id], columns=['0', 'From','ID']) # creates a df by splitting fasta_id list into 3 columns
fasta_id_EntryName:list = fasta_id_df['ID'] # creates a list with the Entry Names from the split df
prot_seq = pd.DataFrame(list(zip(fasta_id_EntryName, fasta_seq)), columns = ["ID", "Sequence"]) # creates a df by combining the Entry Names and the Sequences

# merge dataframes based on Entry Name (df.identifiers) andID (df.prot_seq), if they are similar add the sequence to identifiers, if not add NaN
identifiers_seq = identifiers.merge(prot_seq, how='left', left_on='Entry Name', right_on='ID')[identifiers.columns.tolist() + ['Sequence']] # merges identifiers and prot_seq on Entry Name and ID
#species.head()




In [7]:
#identifiers_seq.head()
identifiers.tail()

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Temperature dependence
34256,R4YVE1,R4YVE1,unreviewed,R4YVE1_OLEAN,Putative ActC family protein,OLEAN_C38630,Oleispira antarctica RB-8,284,NaN,NaN,NaN,NaN
34257,R4YVE4,R4YVE4,unreviewed,R4YVE4_OLEAN,Putative sulfate transport protein CysZ,cysZ OLEAN_C38780,Oleispira antarctica RB-8,244,sulfate transport [GO:0008272],plasma membrane [GO:0005886],NaN,NaN
34258,R4YVE5,R4YVE5,unreviewed,R4YVE5_OLEAN,HlyD family secretion protein,OLEAN_C38830,Oleispira antarctica RB-8,334,NaN,NaN,NaN,NaN
34259,R4YVE9,R4YVE9,unreviewed,R4YVE9_OLEAN,ATP synthase gamma chain (ATP synthase F1 sect...,atpG2 atpG OLEAN_C39030,Oleispira antarctica RB-8,286,proton motive force-driven plasma membrane ATP...,plasma membrane [GO:0005886]; proton-transport...,ATP binding [GO:0005524]; hydrolase activity [...,NaN
34260,R4YVF0,R4YVF0,unreviewed,R4YVF0_OLEAN,ATP synthase subunit a (ATP synthase F0 sector...,atpB2 atpB OLEAN_C39080,Oleispira antarctica RB-8,299,NaN,plasma membrane [GO:0005886]; proton-transport...,hydrolase activity [GO:0016787]; proton-transp...,NaN
